In [1]:
print(spark.version)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8,application_1752432793451_0009,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.5.5-amzn-0

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType, LongType, TimestampNTZType, StringType
import os

spark = SparkSession.builder.master("local[*]").appName("homework3").getOrCreate()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
yellow_path = 's3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow'
green_path =  's3://robot-dreams-source-data/home-work-1/nyc_taxi/green'
lookup_path = 's3://robot-dreams-source-data/home-work-1/nyc_taxi/taxi_zone_lookup.csv'

taxi_schema = StructType([
    StructField("VendorID", LongType(), True),
    StructField("tpep_pickup_datetime", TimestampNTZType(), True),
    StructField("tpep_dropoff_datetime", TimestampNTZType(), True),
    StructField("lpep_pickup_datetime", TimestampNTZType(), True),
    StructField("lpep_dropoff_datetime", TimestampNTZType(), True),
    StructField("passenger_count", LongType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("RatecodeID", LongType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("PULocationID", LongType(), True),
    StructField("DOLocationID", LongType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("congestion_surcharge", DoubleType(), True),
    StructField("airport_fee", IntegerType(), True),
    StructField("ehail_fee", IntegerType(), True),
    StructField("trip_type", DoubleType(), True)
])    


# Schema Harmonization
common_columns = [
    "VendorID",
    "store_and_fwd_flag",
    "PULocationID",
    "DOLocationID",
    "RatecodeID",
    "passenger_count",
    "trip_distance",
    "fare_amount",
    "extra",
    "mta_tax",
    "tip_amount",
    "tolls_amount",
    "improvement_surcharge",
    "total_amount",
    "payment_type",
    "congestion_surcharge"
]

# Read yellow taxi data
#yellow_df_raw = spark.read.option("mergeSchema", "true").option("recursiveFileLookup", "true").schema(taxi_schema).parquet(yellow_path)

paths = [f"{yellow_path}/{year}/" for year in [2014, 2015, 2016, 2017, 2018, 2020, 2022]]
dfs = []
for path in paths:
    df = spark.read.option("mergeSchema", "true").schema(taxi_schema).parquet(path)
    dfs.append(df)

yellow_df_raw = dfs[0]
for df in dfs[1:]:
    yellow_df_raw = yellow_df_raw.unionByName(df)

yellow_df = yellow_df_raw.select(
    *common_columns,
    F.hour("tpep_pickup_datetime").alias("hour"),
    F.date_format("tpep_pickup_datetime", "E").alias("dayofweek"),
    F.round((F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")) / 60, 2).alias("duration_min"),
    F.col("tpep_pickup_datetime").alias("pickup_datetime"),
    F.col("tpep_dropoff_datetime").alias("dropoff_datetime"),
    F.col("PULocationID").cast(LongType()).alias("pickup_location_id"),
    F.col("DOLocationID").cast(LongType()).alias("dropoff_location_id"),
    F.lit("yellow").alias("taxi_type"),
).drop("tpep_pickup_datetime")

# Read green taxi data
#green_df_raw = spark.read.option("mergeSchema", "true").option("recursiveFileLookup", "true").schema(taxi_schema).parquet(green_path)

paths = [f"{green_path}/{year}/" for year in [2014, 2015, 2016, 2017, 2018, 2020, 2022]]
dfs = []
for path in paths:
    df = spark.read.option("mergeSchema", "true").schema(taxi_schema).parquet(path)
    dfs.append(df)

green_df_raw = dfs[0]
for df in dfs[1:]:
    green_df_raw = green_df_raw.unionByName(df)

green_df = green_df_raw.select(
    *common_columns,
    F.hour("lpep_pickup_datetime").alias("hour"),
    F.date_format("lpep_pickup_datetime", "E").alias("dayofweek"),
    F.round((F.unix_timestamp("lpep_pickup_datetime") - F.unix_timestamp("lpep_pickup_datetime")) / 60, 2).alias("duration_min"),
    F.col("lpep_pickup_datetime").alias("pickup_datetime"),
    F.col("lpep_pickup_datetime").alias("dropoff_datetime"),
    F.col("ehail_fee").alias("ehail_fee"),
    F.col("trip_type").alias("trip_type"),
    F.col("PULocationID").cast(LongType()).alias("pickup_location_id"),
    F.col("DOLocationID").cast(LongType()).alias("dropoff_location_id"),
    F.lit("green").alias("taxi_type")
).drop("lpep_pickup_datetime")

raw_trips_df = yellow_df.unionByName(green_df, allowMissingColumns=True)

raw_trips_df.limit(5).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------+------------+------------+----------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+----+---------+------------+-------------------+-------------------+------------------+-------------------+---------+---------+---------+
|VendorID|store_and_fwd_flag|PULocationID|DOLocationID|RatecodeID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|hour|dayofweek|duration_min|    pickup_datetime|   dropoff_datetime|pickup_location_id|dropoff_location_id|taxi_type|ehail_fee|trip_type|
+--------+------------------+------------+------------+----------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+----+---------+------------+-------------------+-------------------+-----------

In [4]:
filtered_trips_df = raw_trips_df.filter((F.col("trip_distance") >= 0.1) \
                                        & (F.col("total_amount") >= 2.0) \
                                        & (F.col("duration_min") >= 1.0) \
                                       )

zones_df = spark.read.option("header", True).csv(lookup_path) \
    .select(
        F.col("LocationID").cast(LongType()).alias("location_id"),
        F.col("zone").alias("zone")
    )

# add pickup_zone, dropoff_zone.
trips_joined_zones_df = filtered_trips_df \
    .join(zones_df, filtered_trips_df.pickup_location_id == zones_df.location_id, "left") \
    .withColumn("pickup_zone", F.when(F.col("zone").isNull(), "Unknown").otherwise(F.col("zone"))) \
    .drop("location_id", "zone") \
    .join(zones_df, filtered_trips_df.dropoff_location_id == zones_df.location_id, "left") \
    .withColumn("dropoff_zone", F.when(F.col("zone").isNull(), "Unknown").otherwise(F.col("zone"))) \
    .drop("location_id", "zone")

trips_joined_zones_df.limit(5).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------+------------+------------+----------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+----+---------+------------+-------------------+-------------------+------------------+-------------------+---------+---------+---------+--------------------+--------------+
|VendorID|store_and_fwd_flag|PULocationID|DOLocationID|RatecodeID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|hour|dayofweek|duration_min|    pickup_datetime|   dropoff_datetime|pickup_location_id|dropoff_location_id|taxi_type|ehail_fee|trip_type|         pickup_zone|  dropoff_zone|
+--------+------------------+------------+------------+----------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+----+--

In [5]:
zone_summary = trips_joined_zones_df.groupBy("pickup_zone").agg(
    F.count(F.lit(1)).alias("total_tips"),
    F.avg("trip_distance").alias("avg_trip_distance"),
    F.avg("total_amount").alias("avg_total_amount"),
    F.avg("tip_amount").alias("avg_tip_amount"),
    F.round(
        F.sum(F.when(F.col("taxi_type") == "yellow", F.lit(1)).otherwise(F.lit(0)))
        / F.sum(F.lit(1))
        ,2)
    .alias("yellow_share"),
    F.round((F.lit(1) - F.col("yellow_share")), 2).alias("green_share"),
    F.max("trip_distance").alias("max_trip_distance"),
    F.min("tip_amount").alias("min_tip_amount")
)

zone_days_statstic = trips_joined_zones_df \
    .withColumn("numdayofweek", F.dayofweek("pickup_datetime")) \
    .groupBy("pickup_zone", "dayofweek", "numdayofweek").agg(
    F.count(F.lit(1)).alias("total_day_count"),
    F.round(
        F.sum(F.when(F.col("total_amount") > 30, F.lit(1)).otherwise(F.lit(0)))
        / F.sum(F.lit(1))
        ,2)
    .alias("high_fare_share")
).orderBy("pickup_zone", "numdayofweek").drop("numdayofweek")

zone_summary.limit(5).show()
zone_days_statstic.limit(5).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+------------------+------------------+------------------+------------+-----------+-----------------+--------------+
|         pickup_zone|total_tips| avg_trip_distance|  avg_total_amount|    avg_tip_amount|yellow_share|green_share|max_trip_distance|min_tip_amount|
+--------------------+----------+------------------+------------------+------------------+------------+-----------+-----------------+--------------+
|         Westerleigh|       589| 8.292750424448213| 39.20227504244481|3.4252631578947357|         1.0|        0.0|            124.4|           0.0|
|Charleston/Totten...|      1299|30.955111624326417| 93.88849884526557|  2.17423402617398|         1.0|        0.0|            103.0|           0.0|
|      Pelham Parkway|      9244| 5.813241021202942| 24.85483881436607|1.2378180441367377|         1.0|        0.0|           104.15|           0.0|
|           Rego Park|     68684| 5.594908275580922|24.053239764719432| 1.963506056723545|         1.0|   

In [7]:
import shutil
import boto3
from datetime import date

# ============saving===========
today_date = date.today().isoformat()
s3 = boto3.client("s3")
bucket = "isilantiev-emr-studio"

def s3_rename(prefix: str, target_name: str):
  source_key = None
  resp = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
  for obj in resp.get("Contents", []):
    key = obj["Key"]
    if key.endswith(".parquet"):
        source_key = key
        break

  if source_key:    
    target_key = target_name
    s3.copy_object(
        Bucket=bucket,
        CopySource={"Bucket": bucket, "Key": source_key},
        Key=target_key
    )
      
    # clean up temp folder
    s3.delete_object(Bucket=bucket, Key=source_key)
  s3.delete_object(Bucket=bucket, Key=prefix + "/_SUCCESS")


# write to s3 both of files
zone_statistic_path = os.path.join("results/zone_statistic", today_date)
zone_summary_single = zone_summary.coalesce(1)
zone_summary_single.write.parquet(f"s3://{bucket}/{zone_statistic_path}", mode="overwrite")
s3_rename(zone_statistic_path, os.path.join(zone_statistic_path, "zone_summary.parquet"))

zone_days_statistic_path = os.path.join("results/zone_days_statstic", today_date)
zone_days_summary_single = zone_days_statstic.coalesce(1)
zone_days_summary_single.write.parquet(f"s3://{bucket}/{zone_days_statistic_path}", mode="overwrite")
s3_rename(zone_days_statistic_path, os.path.join(zone_days_statistic_path,"zone_days_statstic.parquet"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…